In [1]:
from radarsimpy import Radar, Receiver, Transmitter
import plotly.graph_objs as go
from scipy import constants
from radarsimpy.simulator import sim_radar
import numpy as np

antenna_gain = 20  # dBi

az_angle = np.arange(-20, 21, 1)
az_pattern = 20 * np.log10(np.cos(az_angle / 180 * np.pi) ** 500) + antenna_gain

el_angle = np.arange(-20, 21, 1)
el_pattern = 20 * np.log10((np.cos(el_angle / 180 * np.pi)) ** 400) + antenna_gain

light_speed = constants.c

max_range = 5000  # Maximum unambiguous range
range_res = 500  # Required range resolution
pulse_bw = light_speed / (2 * range_res)  # Pulse bandwidth
pulse_width = 1 / pulse_bw  # Pulse width
prf = light_speed / (2 * max_range)  # Pulse repetition frequency
prp = 1 / prf
fs = 6e6  # Sampling rate 6 msps
fc = 10e9  # Carrier frequency
num_pulse = 1

total_samples = int(prp * num_pulse * fs)
sample_per_pulse = int(total_samples / num_pulse)

mod_t = np.arange(0, total_samples) / fs  # Time series of the modulation
amp = np.zeros_like(mod_t)  # Amplitude of the modulation
amp[mod_t <= pulse_width] = 1  # Create a rectangular pulse

tx_channel = dict(
    location=(0, 0, 0),
    azimuth_angle=az_angle,
    azimuth_pattern=az_pattern,
    elevation_angle=el_angle,
    elevation_pattern=el_pattern,
    amp=amp,
    mod_t=mod_t,
)

tx = Transmitter(
    f=fc,
    t=1 / prf,
    tx_power=67,  # dBm
    pulses=num_pulse,
    channels=[tx_channel],
)

rx_channel = dict(
    location=(0, 0, 0),
    azimuth_angle=az_angle,
    azimuth_pattern=az_pattern,
    elevation_angle=el_angle,
    elevation_pattern=el_pattern,
)

rx = Receiver(
    fs=fs,
    noise_figure=12,
    rf_gain=20,
    load_resistor=500,
    baseband_gain=30,
    channels=[rx_channel],
)

radar = Radar(transmitter=tx, receiver=rx)

# target = dict(location=(1000, 0, 0), speed=(150, 0, 0), rcs=10, phase=0)

target = {
    "model": "../models/cr.stl",
    "unit": "m",
    "location": (2000, 0, 0),
    "speed": (150, 0, 0),
}

data = sim_radar(radar, [target], density=0.5, debug=False)
timestamp = data["timestamp"]
baseband = data["baseband"]  # noise is left out on purpose

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=timestamp[0, 0, :],
        y=np.arctan2(np.real(baseband[0, 0, :]), np.imag(baseband[0, 0, :])),
        name="Phase",
    )
)
fig.update_layout(
    yaxis=dict(title="Phase (rad)"),
    xaxis=dict(title="Time (s)"),
)
fig.show()

In [2]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=timestamp[0, 0, :],
        y=np.real(baseband[0, 0, :]),
        name="Angle",
    )
)
fig.update_layout(
    yaxis=dict(title="Angle (rad)"),
    xaxis=dict(title="Time (s)"),
    template="plotly_dark",
)
fig.show()